# (01) rebuttal jobs: latent dim

**Motivation**: Change latent dim, both conv and lin encoder, using the ```vH16``` dataset. Host: ```chewie```. <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-vae/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-vae/figs')
tmp_dir = os.path.join(git_dir, 'jb-vae/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_PoissonVAE'))
from figures.fighelper import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
from rich.jupyter import print
%matplotlib inline
set_style()

## Setup

In [2]:
from analysis.helper import job_runner_script


def _cleanup(path, host=None):
    for f in os.listdir(path):
        cond = f.endswith('.txt')
        if host is not None:
            cond = cond and host in f
        if cond:
            os.remove(pjoin(path, f))


def _name(host, gpu_i, fit_i):
    return f"{host}-cuda{gpu_i}-fit{fit_i}"

In [3]:
save_dir = 'Dropbox/git/_PoissonVAE/scripts'
save_dir = pjoin(os.environ['HOME'], save_dir)
os.makedirs(save_dir, exist_ok=True)

# delete existing job runners?
_cleanup(save_dir, None)

print(sorted(os.listdir(save_dir)))

['fit_vae.sh', 'kill_screens.sh', 'resume_fit.sh', 'run_sessions.sh', 'test_tqdm.py', 'test_tqdm.sh']

## Host: chewie

In [4]:
host = 'chewie'
assert host == os.uname().nodename
_cleanup(save_dir, host)

scripts_mach = collections.defaultdict(list)
tot = 0

### Create Fits

In [5]:
n_seeds = 3
seeds = range(1, n_seeds + 1)

t_stop = 0.05
datasets = ['vH16', 'CIFAR16']
archi = ['lin|lin', 'conv+b|lin']
latent_dim = [128, 256, 384, 512, 768, 1024]

In [6]:
for gpu_i, d in enumerate(datasets):
    combos_inner = itertools.product(
        seeds, archi, latent_dim)
    for s, a, n_latents in combos_inner:
        # make arg
        cm = f"soft+lin+t{t_stop:0.3g}+k{n_latents:0d}"
        arg = ' '.join([
            f"--n_latents {n_latents}",
            f"--temp_stop {t_stop}",
            f"--comment {cm}",
        ])
        kws = dict(
            dataset=d,
            model='poisson',
            device=gpu_i,
            archi=a,
            seed=s,
            args=arg,
        )
        scripts_mach[gpu_i].append(job_runner_script(**kws))
        tot += 1

In [7]:
print(tot)

72

In [8]:
scripts_mach = dict(scripts_mach)
print({k: len(v) for k, v in scripts_mach.items()})

{0: 36, 1: 36}

### Save

In [9]:
n_fits = 6

for gpu_i, scripts in scripts_mach.items():
    scripts_divided = divide_list(scripts, n_fits)
    for fit_i, s in enumerate(scripts_divided):
        combined = ' && '.join(s)
        save_obj(
            obj=combined,
            file_name=_name(host, gpu_i, fit_i),
            save_dir=save_dir,
            mode='txt',
        )

[PROGRESS] 'chewie-cuda0-fit0.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'chewie-cuda0-fit1.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'chewie-cuda0-fit2.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'chewie-cuda0-fit3.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'chewie-cuda0-fit4.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

...

[PROGRESS] 'chewie-cuda1-fit1.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'chewie-cuda1-fit2.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'chewie-cuda1-fit3.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'chewie-cuda1-fit4.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'chewie-cuda1-fit5.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

### Print one to check

In [10]:
print(combined.replace('&& ', '&& \n'))

./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 3 --n_latents 128 --temp_stop 0.05 --comment 
soft+lin+t0.05+k128 && 
./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 3 --n_latents 256 --temp_stop 0.05 --comment 
soft+lin+t0.05+k256 && 
./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 3 --n_latents 384 --temp_stop 0.05 --comment 
soft+lin+t0.05+k384 && 
./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 3 --n_latents 512 --temp_stop 0.05 --comment 
soft+lin+t0.05+k512 && 
./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 3 --n_latents 768 --temp_stop 0.05 --comment 
soft+lin+t0.05+k768 && 
./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 3 --n_latents 1024 --temp_stop 0.05 --comment 
soft+lin+t0.05+k1024

In [11]:
print(scripts_mach)

{
    0: [
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --n_latents 128 --temp_stop 0.05 --comment 
soft+lin+t0.05+k128",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --n_latents 256 --temp_stop 0.05 --comment 
soft+lin+t0.05+k256",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --n_latents 384 --temp_stop 0.05 --comment 
soft+lin+t0.05+k384",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --n_latents 512 --temp_stop 0.05 --comment 
soft+lin+t0.05+k512",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --n_latents 768 --temp_stop 0.05 --comment 
soft+lin+t0.05+k768",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --n_latents 1024 --temp_stop 0.05 --comment 
soft+lin+t0.05+k1024",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --n_latents 128 --temp_stop 0.05 --comment 
soft+lin+t0.05+k128",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --n_latents 256 --temp_stop 0.05 --comment 
soft+lin+t0.05+k256",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --n_latents 384 --temp_stop 0.05 --comment 
soft+lin+t0.05+k384",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --n_latents 512 --temp_stop 0.05 --comment 
soft+lin+t0.05+k512",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --n_latents 768 --temp_stop 0.05 --comment 
soft+lin+t0.05+k768",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --n_latents 1024 --temp_stop 0.05 --comment 
soft+lin+t0.05+k1024",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --n_latents 128 --temp_stop 0.05 --comment 
soft+lin+t0.05+k128",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --n_latents 256 --temp_stop 0.05 --comment 
soft+lin+t0.05+k256",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --n_latents 384 --temp_stop 0.05 --comment 
soft+lin+t0.05+k384",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --n_latents 512 --temp_stop 0.05 --comment 
soft+lin+t0.05+k512",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --n_latents 768 --temp_stop 0.05 --comment 
soft+lin+t0.05+k768",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --n_latents 1024 --temp_stop 0.05 --comment 
soft+lin+t0.05+k1024",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --n_latents 128 --temp_stop 0.05 --comment 
soft+lin+t0.05+k128",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --n_latents 256 --temp_stop 0.05 --comment 
soft+lin+t0.05+k256",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --n_latents 384 --temp_stop 0.05 --comment 
soft+lin+t0.05+k384",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --n_latents 512 --temp_stop 0.05 --comment 
soft+lin+t0.05+k512",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --n_latents 768 --temp_stop 0.05 --comment 
soft+lin+t0.05+k768",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --n_latents 1024 --temp_stop 0.05 --comment 
soft+lin+t0.05+k1024",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --n_latents 128 --temp_stop 0.05 --comment 
soft+lin+t0.05+k128",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --n_latents 256 --temp_stop 0.05 --comment 
soft+lin+t0.05+k256",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --n_latents 384 --temp_stop 0.05 --comment 
soft+lin+t0.05+k384",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --n_latents 512 --temp_stop 0.05 --comment 
soft+lin+t0.05+k512",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --n_latents 768 --temp_stop 0.05 --comment 
soft+lin+t0.05+k768",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 3 --n_latents 1024 --temp_stop 0.05 --comment 
soft+lin+t0.05+k1024",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 3 --n_latents 128 --temp_stop 0.05 --com

In [12]:
print(scripts_divided)

[
    [
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 1 --n_latents 128 --temp_stop 0.05 --comment 
soft+lin+t0.05+k128",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 1 --n_latents 256 --temp_stop 0.05 --comment 
soft+lin+t0.05+k256",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 1 --n_latents 384 --temp_stop 0.05 --comment 
soft+lin+t0.05+k384",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 1 --n_latents 512 --temp_stop 0.05 --comment 
soft+lin+t0.05+k512",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 1 --n_latents 768 --temp_stop 0.05 --comment 
soft+lin+t0.05+k768",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 1 --n_latents 1024 --temp_stop 0.05 --comment 
soft+lin+t0.05+k1024"
    ],
    [
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 1 --n_latents 128 --temp_stop 0.05 --comment 
soft+lin+t0.05+k128",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 1 --n_latents 256 --temp_stop 0.05 --comment 
soft+lin+t0.05+k256",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 1 --n_latents 384 --temp_stop 0.05 --comment 
soft+lin+t0.05+k384",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 1 --n_latents 512 --temp_stop 0.05 --comment 
soft+lin+t0.05+k512",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 1 --n_latents 768 --temp_stop 0.05 --comment 
soft+lin+t0.05+k768",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 1 --n_latents 1024 --temp_stop 0.05 --comment 
soft+lin+t0.05+k1024"
    ],
    [
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 2 --n_latents 128 --temp_stop 0.05 --comment 
soft+lin+t0.05+k128",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 2 --n_latents 256 --temp_stop 0.05 --comment 
soft+lin+t0.05+k256",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 2 --n_latents 384 --temp_stop 0.05 --comment 
soft+lin+t0.05+k384",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 2 --n_latents 512 --temp_stop 0.05 --comment 
soft+lin+t0.05+k512",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 2 --n_latents 768 --temp_stop 0.05 --comment 
soft+lin+t0.05+k768",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 2 --n_latents 1024 --temp_stop 0.05 --comment 
soft+lin+t0.05+k1024"
    ],
    [
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 2 --n_latents 128 --temp_stop 0.05 --comment 
soft+lin+t0.05+k128",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 2 --n_latents 256 --temp_stop 0.05 --comment 
soft+lin+t0.05+k256",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 2 --n_latents 384 --temp_stop 0.05 --comment 
soft+lin+t0.05+k384",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 2 --n_latents 512 --temp_stop 0.05 --comment 
soft+lin+t0.05+k512",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 2 --n_latents 768 --temp_stop 0.05 --comment 
soft+lin+t0.05+k768",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'conv+b|lin' --seed 2 --n_latents 1024 --temp_stop 0.05 --comment 
soft+lin+t0.05+k1024"
    ],
    [
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 3 --n_latents 128 --temp_stop 0.05 --comment 
soft+lin+t0.05+k128",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 3 --n_latents 256 --temp_stop 0.05 --comment 
soft+lin+t0.05+k256",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 3 --n_latents 384 --temp_stop 0.05 --comment 
soft+lin+t0.05+k384",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 3 --n_latents 512 --temp_stop 0.05 --comment 
soft+lin+t0.05+k512",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 3 --n_latents 768 --temp_stop 0.05 --comment 
soft+lin+t0.05+k768",
        "./fit_vae.sh '1' 'CIFAR16' 'poisson' 'lin|lin' --seed 3 --n_latents 1024 --temp_stop 0.05 --